In [1]:
import os 
import os.path as osp
os.chdir(osp.dirname(os.getcwd()))
os.getcwd()

'/home/DISCOVER_summer2022/xusc/exp/butd_detr'

In [2]:
from loguru import logger
from my_script.pc_utils import *
from my_script.utils import *
from src.joint_det_dataset import Joint3DDataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2022-10-21 14:18:32,702 - collision - No FCL -- collision checking will not work
WARNING - 2022-10-21 14:18:32,828 - svg_io - SVG path loading unavailable!


In [10]:
train_dataset = Joint3DDataset(
            # dataset_dict={'sr3d':1,'scannet':10},
            dataset_dict={'scannet':10},
            test_dataset='sr3d', #? only test set need ? 
            split='train',
            use_color=True, use_height=False,
            overfit=True,
            data_path='datasets/',
            detect_intermediate=True,#? 
            use_multiview=False, #? 
            butd=False,
            butd_gt=False,
            butd_cls=True,
            augment_det=False
        )

Loading train files, take a breath!


In [11]:
length = len(train_dataset.annos)
for idx in range(length):
    data = train_dataset.__getitem__(idx)
    if len(data['utterances'].split('.'))>4:
        break
    

In [12]:

prompt_demo_idx = 128
train_dataset.__getitem__(prompt_demo_idx)['utterances']
get_attr(data)
print_attr_shape(data)
print(f" box num : {(data['box_label_mask']==1).sum()}")
print(f" label number :  {len(data['utterances'].split('.'))}")
#todo : label 如何映射的? 
#todo :  utterances : 如何设计的 ?  如何设计positive and negtive
#* 


2022-10-21 14:49:53.535 | INFO     | my_script.utils:print_attr_shape:301 - box_label_mask : (132,)
2022-10-21 14:49:53.536 | INFO     | my_script.utils:print_attr_shape:301 - center_label : (132, 3)
2022-10-21 14:49:53.536 | INFO     | my_script.utils:print_attr_shape:301 - sem_cls_label : (132,)
2022-10-21 14:49:53.537 | INFO     | my_script.utils:print_attr_shape:301 - size_gts : (132, 3)
2022-10-21 14:49:53.538 | INFO     | my_script.utils:print_attr_shape:295 - scan_ids : scene0191_00
2022-10-21 14:49:53.538 | INFO     | my_script.utils:print_attr_shape:301 - point_clouds : (50000, 6)
2022-10-21 14:49:53.539 | INFO     | my_script.utils:print_attr_shape:295 - utterances : cabinet . bed . chair . couch . table . door . window . bookshelf . picture . counter . desk . curtain . refrigerator . shower curtain . toilet . sink . bathtub . other furniture . not mentioned
2022-10-21 14:49:53.539 | INFO     | my_script.utils:print_attr_shape:301 - tokens_positive : (132, 2)
2022-10-21 14:49

 box num : 7
 label number :  19


In [16]:


def get_caption(utterances):
    caption = ' '.join(utterances.replace(',', ' ,').split())
    caption = ' ' + caption + ' '

    
    return caption
'''
description: 根据tokens_positive 从utterances 取text token 
param {*} utterances
param {*} tokens_positive
return {*}
'''
def get_token(utterances,tokens_positive):

    caption = get_caption(utterances)
    all_text_token_from_utterance = []
    for start,end in tokens_positive:
        tmp = caption[start:end+1].strip()
        all_text_token_from_utterance.append(tmp)
        # print(tmp,end='\t')
    return all_text_token_from_utterance


logger.info(data['utterances'])
all_labels = data['utterances'].split('.')
logger.info(f" utterances length : {len(all_labels)}")

# all_labels
#?  negtive label 为什么会有box 与之对应呢?    多出来的两个box 是什么?  应该是与negtive sample对应的... 
all_box = data['all_bboxes'][data['all_bbox_label_mask']]
logger.info(f" box  number in the scene  : {len(all_box)}")
all_target_box = data['center_label'][data['box_label_mask']==1]
logger.info(f"referred tagret box number  in the scene  : {all_target_box.shape[0]}") 


logger.info(f"referred target object sem_cls_label: {data['sem_cls_label'][data['box_label_mask']==1]}")#* 这个没用吗? 为什么sem_cls_label 都是对应0 ? 

# logger.info(f"data['positive_map'].shape {data['positive_map'].shape}")#* 这个跟 language model 的输出有关
all_text_token_from_potive_token = get_token(data['utterances'],data['tokens_positive'][data['box_label_mask']==1])

text_from_utterance_appear_in_positive_token = []
for label in all_labels:
    if label.strip() in all_text_token_from_potive_token:
    
        text_from_utterance_appear_in_positive_token.append(label)
logger.info(f"  {len(text_from_utterance_appear_in_positive_token)} object in the utterance is referred ")
logger.info(f"all text token referred as postive token  {all_text_token_from_potive_token}")
logger.info(f" {len(all_text_token_from_potive_token)} referred as postive token")
#* 用utterances 去找只能找到  10个, 而referred target有15个, 也就是说有5个重复的目标, 


logger.info(f"data['distractor_ids'] : {data['distractor_ids']},length : {len(data['distractor_ids'])}")
logger.info(f",data['anchor_ids'] : {data['anchor_ids']},len : {len( data['anchor_ids'])}")
logger.info(f"data['target_cid'] : {data['target_cid']}")








2022-10-21 14:52:20.817 | INFO     | __main__:<module>:24 - cabinet . bed . chair . couch . table . door . window . bookshelf . picture . counter . desk . curtain . refrigerator . shower curtain . toilet . sink . bathtub . other furniture . not mentioned
2022-10-21 14:52:20.819 | INFO     | __main__:<module>:26 -  utterances length : 19
2022-10-21 14:52:20.820 | INFO     | __main__:<module>:31 -  box  number in the scene  : 17
2022-10-21 14:52:20.821 | INFO     | __main__:<module>:33 - referred tagret box number  in the scene  : 7
2022-10-21 14:52:20.822 | INFO     | __main__:<module>:36 - referred target object sem_cls_label: [ 5  4  6 17  2  6  6]
2022-10-21 14:52:20.824 | INFO     | __main__:<module>:46 -   5 object in the utterance is referred 
2022-10-21 14:52:20.825 | INFO     | __main__:<module>:47 - all text token referred as postive token  ['door', 'table', 'window', 'other furniture', 'chair', 'window', 'window']
2022-10-21 14:52:20.825 | INFO     | __main__:<module>:48 -  7 

In [8]:
data['all_bboxes'][data['all_bbox_label_mask']].shape

(17, 6)

In [9]:
data['all_class_ids'][data['all_bbox_label_mask']]

array([  0,   0,   4,   3,  62,  62,  14,  42,  50,   1,  14, 325,  46,
        14,  46,   2,   7])